# Import the Libsbml library

In [2]:
import libsbml

## Read in the model

In [23]:
r = libsbml.SBMLReader()
d = r.readSBML('R-HSA-168255.sbml')
m = d.getModel()
m

<Model pathway_168255 "Influenza Infection">

In [24]:
m.getNumReactions()

59

In [25]:
m.getNumSpecies()

142

In [11]:
print(m.notes.toXMLString())

<notes>
  <p xmlns="http://www.w3.org/1999/xhtml">For centuries influenza epidemics have plagued man; with influenza probably being the disease described by Hippocrates in 412 BC. Today it remains a major cause of morbidity and mortality worldwide with large segments of the human population affected every year. Many animal species can be infected by influenza viruses, often with catastrophic consequences. An influenza pandemic is a continuing global level threat. The 1918 influenza pandemic is a modern example of how devastating such an event could be with an estimated 50 million deaths worldwide. Influenza viruses belong to the family of Orthomyxoviridae; viruses with segmented RNA genomes that are negative sense and single-stranded (Baltimore 1971). Influenza virus strains are named according to their type (A, B, or C), the species from which the virus was isolated (omitted if human), location of isolate, the number of the isolate, the year of isolation, and in the case of influenza 

In [14]:

print('\n'.join(['\t'.join([rxn.getId(), rxn.getName()]) 
                 for rxn in m.getListOfReaction()]))

reaction_192677	Newly synthesized vRNP for export
reaction_192746	Binding of M1 to vRNP
reaction_168893	Binding of NEP/NS2 to vRNP:M1
reaction_168880	vRNP Export through the nuclear pore
reaction_168857	Binding of vRNP:M1:NEP complex to CRM1 export receptor
reaction_168285	Clathrin-Mediated Pit Formation And Endocytosis Of The Influenza Virion
reaction_168272	Binding of the influenza virion to the host cell
reaction_192704	Synthesis of PB1-F2
reaction_192841	Viral Protein Synthesis
reaction_192627	Viral mRNA Export
reaction_168326	Assembly of an Active Transcription Complex
reaction_168280	Priming and Initiation of Transcription
reaction_168301	Elongation, Polyadenylation and Termination
reaction_192781	Viral mRNA Splicing (M, NS segments)
reaction_192925	Export of Spliced Viral mRNA
reaction_192832	Initiation of cRNA Synthesis
reaction_192624	cRNA Extension
reaction_192912	NP binds vRNA
reaction_192830	Viral Polymerase Assembly
reaction_192851	vRNA Extension
reaction_192916	Initiation

In [26]:
rxn = m.getReaction('reaction_169919')
reactants = rxn.getListOfReactants()
reactants[0].getSpecies()

'species_203497'

In [28]:
m.getSpecies(reactants[0].getSpecies())

<Species species_203497 "NA [endoplasmic reticulum membrane]">

In [29]:
for rxn in m.getListOfReactions():
    print("{}\t{}".format(rxn.getId(), rxn.getName()))
    print("   Reactants:")
    for reactant in rxn.getListOfReactants():
        specie = m.getSpecies(reactant.getSpecies())
        print("\t{}\t{}".format(specie.getId(), specie.getName()))
    print("   Products:")
    for product in rxn.getListOfProducts():
        specie = m.getSpecies(product.getSpecies())
        print("\t{}\t{}".format(specie.getId(), specie.getName()))
    print("   Regulator:")
    for catalyst in rxn.getListOfModifiers():
        specie = m.getSpecies(catalyst.getSpecies())
        print("\t{}\t{}".format(specie.getId(), specie.getName()))

reaction_192677	Newly synthesized vRNP for export
   Reactants:
	species_188877	PA [nucleoplasm]
	species_188857	PB2 [nucleoplasm]
	species_188843	PB1 [nucleoplasm]
	species_193302	vRNA (Genomic):NP Complex [nucleoplasm]
   Products:
	species_192922	vRNP destined for Export [nucleoplasm]
   Regulator:
	species_188832	NP [nucleoplasm]
reaction_192746	Binding of M1 to vRNP
   Reactants:
	species_186959	M1 [nucleoplasm]
	species_192922	vRNP destined for Export [nucleoplasm]
   Products:
	species_192906	vRNP:M1 for Export [nucleoplasm]
   Regulator:
	species_6792605	HSPA1A [nucleoplasm]
reaction_168893	Binding of NEP/NS2 to vRNP:M1
   Reactants:
	species_192906	vRNP:M1 for Export [nucleoplasm]
	species_169474	NEP/NS2 [nucleoplasm]
   Products:
	species_192767	vRNP:M1:NEP [nucleoplasm]
   Regulator:
reaction_168880	vRNP Export through the nuclear pore
   Reactants:
	species_192667	vRNP Export Complex [nucleoplasm]
	species_180738	RAN:GTP [nucleoplasm]
   Products:
	species_29372	Pi [cytosol

In [22]:
from collections import defaultdict
from functools import partial
from torch import tensor

import statistics
import pyro
from pyro import condition, do, sample
from pyro.optim import SGD
import torch.distributions.constraints as constraints

from pyro.distributions import Normal, Delta
from pyro.infer import EmpiricalMarginal, Importance, SVI, Trace_ELBO
